## 1. Import Packages

Below, we import both standard packages, and functions from the accompanying .py files

In [2]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

#Import metrics
from metrics import get_R2
from metrics import get_rho

#Import decoder functions
from decoders import WienerCascadeDecoder
from decoders import WienerFilterDecoder
from decoders import DenseNNDecoder
from decoders import SimpleRNNDecoder
from decoders import GRUDecoder
from decoders import LSTMDecoder
from decoders import XGBoostDecoder

## 2. Load Data

The data that we load is in the format described below. We have another example script, "neural_preprocessing.py" that may be helpful towards putting the data in this format.

Neural data should be a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin

The output you are decoding should be a matrix of size "number of time bins" x "number of features you are decoding"

In [3]:
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/'
# folder='/Users/jig289/Dropbox/Grad_School/Research/Projects/In_Progress/Decoding/DataFiles/'
folder='/home/jglaser2/Data/DecData/'

with open(folder+'s1_test_data.pickle','rb') as f:
    neural_data,vels_binned,pos_binned,acc_binned=pickle.load(f)

## 3. Preprocess Data

### 3A. User Inputs
The user can define what time period to use spikes from (with respect to the output).

In [4]:
bins_before=13 #How many bins of neural data prior to the output are used for decoding
bins_current=1 #Whether to use concurrent time bin of neural data
bins_after=0 #How many bins of neural data after (and including) the output are used for decoding

### 3B. Format Covariates

#### Format Input Covariates

In [5]:
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
#Put in "flat" format, so each "neuron / time" is a single feature
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

#### Format Output Covariates

In [6]:
#Set decoding output
y=vels_binned

### 3C. Process Covariates
We normalize (z_score) the inputs and zero-center the outputs.

In [7]:
#Z-scoring function that works with Nans:
def zscore_nan(X,axis):
    X_zscore=(X - np.nanmean(X,axis=axis)) / np.nanstd(X,axis=axis)
    return X_zscore

In [8]:
#Normalize inputs
X=zscore_nan(X,axis=0)
X_flat=zscore_nan(X_flat,axis=0)

#Zero-center outputs
y_mean=np.mean(y,axis=0)
y=y-y_mean

### 3D. Split into training/testing sets

#### User Options

In [9]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.5]
testing_range=[0.5, 1]

#### Split Data

In [10]:
num_examples=X.shape[0]

#Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
#This makes it so that the different sets don't include overlapping neural data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples))+bins_before,np.int(np.round(training_range[1]*num_examples))-bins_after)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples))+bins_before,np.int(np.round(testing_range[1]*num_examples))-bins_after)

#Get training data
X_train=X[training_set,:,:]
X_flat_train=X_flat[training_set,:]
y_train=y[training_set,:]

#Get testing data
X_test=X[testing_set,:,:]
X_flat_test=X_flat[testing_set,:]
y_test=y[testing_set,:]

## 4. Train Decoders

### 4A. Wiener Filter (Linear Regression)

In [11]:
#Declare model
model_wf=WienerFilterDecoder()
#Fit model
model_wf.fit(X_flat_train,y_train)

### 4B. Wiener Cascade (Linear Nonlinear Model)

In [12]:
#Declare model
model_wc=WienerCascadeDecoder(degree=5)
#Fit model
model_wc.fit(X_flat_train,y_train)

### 4C. XGBoost (Extreme Gradient Boosting)

In [13]:
#Declare model
model_xgb=XGBoostDecoder(max_depth=2,num_round=1000)
#Fit model
model_xgb.fit(X_flat_train, y_train)

### 4D. Dense Neural Network

In [30]:
#Declare model
model_dnn=DenseNNDecoder(units=[400,400],dropout=0,num_epochs=15)
#Fit model
model_dnn.fit(X_flat_train,y_train)

### 4E. Simple RNN

In [15]:
#Declare model
model_rnn=SimpleRNNDecoder(units=400,dropout=0,num_epochs=10)
#Fit model
model_rnn.fit(X_train,y_train)

### 4F. GRU (Gated Recurrent Unit)

In [16]:
#Declare model
model_gru=GRUDecoder(units=400,dropout=.25,num_epochs=5)
#Fit model
model_gru.fit(X_train,y_train)

### 4G. LSTM (Long Short Term Memory)

In [17]:
#Declare model
model_lstm=LSTMDecoder(units=400,dropout=0,num_epochs=15)
#Fit model
model_lstm.fit(X_train,y_train)

## 5. Save Model Fits

In [18]:
# with open('s1_models.pickle','wb') as f:
#     pickle.dump([model_wf,model_wc,model_xgb,model_dnn,model_rnn,model_gru,model_lstm],f)

## 6. Get Fits on Test Data

In [31]:
num_tests=10
num_test_examples=y_test.shape[0]
num_examples_per_test=np.round(np.divide(num_test_examples,num_tests))

#Initialize
mean_R2_wf=np.empty(num_tests)
mean_R2_wc=np.empty(num_tests)
mean_R2_xgb=np.empty(num_tests)
mean_R2_dnn=np.empty(num_tests)
mean_R2_rnn=np.empty(num_tests)
mean_R2_gru=np.empty(num_tests)
mean_R2_lstm=np.empty(num_tests)

for i in range(num_tests):

    idx=np.arange(num_examples_per_test*i,num_examples_per_test*(i+1))
    X_test_temp=X_test[idx,:,:]
    X_flat_test_temp=X_flat_test[idx,:]
    y_test_temp=y_test[idx,:]

    #Wiener Filter
    y_predicted_wf=model_wf.predict(X_flat_test_temp)
    mean_R2_wf[i]=np.mean(get_R2(y_test_temp,y_predicted_wf))
    
    #Wiener Cascade
    y_predicted_wc=model_wc.predict(X_flat_test_temp)
    mean_R2_wc[i]=np.mean(get_R2(y_test_temp,y_predicted_wc))   
    
    #XGBoost
    y_predicted_xgb=model_xgb.predict(X_flat_test_temp)
    mean_R2_xgb[i]=np.mean(get_R2(y_test_temp,y_predicted_xgb))        
    
    #DNN
    y_predicted_dnn=model_dnn.predict(X_flat_test_temp)
    mean_R2_dnn[i]=np.mean(get_R2(y_test_temp,y_predicted_dnn))   
    
    #RNN
    y_predicted_rnn=model_rnn.predict(X_test_temp)
    mean_R2_rnn[i]=np.mean(get_R2(y_test_temp,y_predicted_rnn))
    
    #GRU
    y_predicted_gru=model_gru.predict(X_test_temp)
    mean_R2_gru[i]=np.mean(get_R2(y_test_temp,y_predicted_gru))
    
    #LSTM
    y_predicted_lstm=model_lstm.predict(X_test_temp)
    mean_R2_lstm[i]=np.mean(get_R2(y_test_temp,y_predicted_lstm))

In [32]:
print(np.mean(mean_R2_wf),np.std(mean_R2_wf)/np.sqrt(10))
print(np.mean(mean_R2_wc),np.std(mean_R2_wc)/np.sqrt(10))
print(np.mean(mean_R2_xgb),np.std(mean_R2_xgb)/np.sqrt(10))
print(np.mean(mean_R2_dnn),np.std(mean_R2_dnn)/np.sqrt(10))
print(np.mean(mean_R2_rnn),np.std(mean_R2_rnn)/np.sqrt(10))
print(np.mean(mean_R2_gru),np.std(mean_R2_gru)/np.sqrt(10))
print(np.mean(mean_R2_lstm),np.std(mean_R2_lstm)/np.sqrt(10))

(0.67550371087286465, 0.0086567180384587699)
(0.69255356886578234, 0.0071501963754975074)
(0.71860100341136934, 0.0071469434400783394)
(0.66458018275342634, 0.01170739267412285)
(0.74256033941068267, 0.0057448139449299759)
(0.78488981144494852, 0.0058230462032647099)
(0.7871825682066883, 0.005618281590838793)


In [34]:
save_folder='/home/jglaser2/Data/Decoding_Results/'
with open(save_folder+'s1_results.pickle','wb') as f:
    pickle.dump([mean_R2_wf,mean_R2_wc,mean_R2_xgb,mean_R2_dnn,mean_R2_rnn,mean_R2_gru,mean_R2_lstm],f)